In [1]:
!pip install googletrans==4.0.0-rc1

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# from transformers import BertTokenizer, BertForSequenceClassification
import pandas as pd
import numpy as np
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
# import torch
# from transformers import TrainingArguments, Trainer
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download("stopwords")
import ast

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\zberl\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
def clean_text(text):
    # text = text.lower()  # Konversi ke lowercase
    text = re.sub(r"(?i)<\s*username\s*>", "", text)  # Hapus tag username
    text = re.sub(r"http\S+|www\S+", "", text)  # Hapus URL
    text = re.sub(r"\s*@\S+", "", text) # Hapus mention
    text = re.sub(r"[^\w\s]", " ", text)  # Hapus karakter khusus (kecuali spasi)
    text = re.sub(r"\s+", " ", text).strip()  # Hapus spasi berlebih

    # Hapus stopwords
    # stop_words = set(stopwords.words("indonesian"))
    # print(stop_words)
    # words = text.split()
    # filtered_words = [word for word in words if word not in stop_words]

    # # Lakukan lemmatization pada setiap kata Inggris
    # lemmatized_words = [lemmatizer.lemmatize(word) for word in filtered_words]

    # # Gabungkan kembali kata-kata hasil stemming
    # text = " ".join(lemmatized_words)

    # #Tokenization kalimat (pisah per kata)
    # tokens = word_tokenize(text)
    # return " ".join(filtered_words)
    return text

In [ ]:
stop_words = set(stopwords.words("indonesian"))
print(stop_words)

{'bersiap', 'suatu', 'wah', 'diucapkannya', 'seolah', 'bekerja', 'sejumlah', 'janganlah', 'waktu', 'benarlah', 'lanjutnya', 'sendirian', 'nanti', 'menjadi', 'benar', 'menjelaskan', 'asalkan', 'bermula', 'seorang', 'haruslah', 'memerlukan', 'melihatnya', 'banyak', 'menanti-nanti', 'tertuju', 'sebisanya', 'soal', 'padanya', 'yaitu', 'itu', 'makanya', 'apabila', 'ditunjuk', 'jikalau', 'siapakah', 'begitulah', 'dilalui', 'siap', 'minta', 'jelaslah', 'memperlihatkan', 'ditanya', 'saja', 'sebuah', 'beginian', 'dimulailah', 'lama', 'antar', 'ditanyakan', 'inilah', 'demi', 'diingatkan', 'mengapa', 'memisalkan', 'dimaksudkannya', 'kalaupun', 'mulailah', 'antara', 'apa', 'bawah', 'sekurangnya', 'dikarenakan', 'kepadanya', 'per', 'sesaat', 'seberapa', 'se', 'jelaskan', 'dikira', 'sama', 'dia', 'selamanya', 'ditujukan', 'bukankah', 'sebegitu', 'dimintai', 'memastikan', 'terakhir', 'para', 'sekiranya', 'diperbuat', 'rasa', 'akankah', 'enggaknya', 'sepanjang', 'bermacam-macam', 'keinginan', 'penting

In [ ]:
def load_model():
    # Load dataset
    df = pd.read_excel("/content/(shuffled) Dataset_Gabungan_with_Data_Augmented.xlsx", index_col=False)
    # df = pd.read_excel("/content/(shuffled) Dataset_Gabungan.xlsx", index_col=False)
    # df = pd.read_excel("/content/Dataset A (1).xlsx", index_col=False)
    # df = pd.read_excel("/content/Dataset B (1).xlsx", index_col=False)

    print(df)
    # Hapus kolom Id
    if "Id" in df.columns:
        df.drop(columns=["Id"], inplace=True)

    # Konversi label sentimen ke numerik
    df["Sentiment"] = df["Sentiment"].map({"negative": 0, "positive": 1})
    # df["Sentiment"] = df["Kategori"].map({"Non-bullying": 1, "Bullying": 0})


    # Bersihkan teks komentar
    # df["Cleaned_Text"] = df["Instagram Comment Text"]
    df["Cleaned_Text"] = df["Instagram Comment Text"].apply(clean_text)
    # df["Cleaned_Text"] = df["Komentar"].apply(clean_text)


    # Buat dataset final
    dataset = pd.DataFrame({"Text": df["Cleaned_Text"],"Label": df["Sentiment"]})

    return dataset

In [ ]:
dataset = load_model()
print(dataset)
dataset.to_excel("dataset_baru.xlsx")

        Id Sentiment                             Instagram Comment Text
0        1  negative          "Udah ada berlian malah milih batu nisan"
1        2  negative          "2 3 tutup botol muka lu kek kontol wkwk"
2        3  negative  "Mukalu kek aktor bokep bgst mending ngobat aj...
3        4  positive  Hatinya terbuat dari apa ya .. Mbaa gigi yang ...
4        5  positive                "Motivasi ku ka Maudy cantipp bngt"
...    ...       ...                                                ...
1745  1746  positive  Selalu suka sama via org nya jujur apa adanya ...
1746  1747  positive  Selalu suka sama via org nya jujur apa adanya ...
1747  1748  positive  Selalu suka sama via org nya jujur apa adanya ...
1748  1749  positive  Selalu suka sama via org nya jujur apa adanya ...
1749  1750  positive  Selalu suka sama via org nya jujur apa adanya ...

[1750 rows x 3 columns]
                                                   Text  Label
0               Udah ada berlian malah milih bat

In [ ]:
def text_to_bio(text):
    bio_tags = []
    in_token = False  # Track if inside a token

    for i, char in enumerate(text):
        if char.isalnum():  # Consider letters/numbers as part of tokens
            if not in_token:  # If not in a token, mark it as 'B'
                bio_tags.append('B')
                in_token = True
            else:  # Otherwise, it's 'I'
                bio_tags.append('I')
        else:  # Non-alphanumeric characters are 'O'
            bio_tags.append('O')
            in_token = False  # Reset token flag

    return bio_tags

# Example
text = "Semakin glowing say… natural kayak gini bagus tambahin prestasi nya ya say"
bio_tags = text_to_bio(text)
print("Text: ", text)
print("BIO:  ", "".join(bio_tags))

Text:  Semakin glowing say… natural kayak gini bagus tambahin prestasi nya ya say
BIO:   BIIIIIIOBIIIIIIOBIIOOBIIIIIIOBIIIIOBIIIOBIIIIOBIIIIIIIOBIIIIIIIOBIIOBIOBII


In [ ]:
def char_features(text, i):
    char = text[i]

    features = {
        'char': char,
        'is_upper': char.isupper(), #dia kapital atau ga, kalau kapital kemungkinan besar awal kata
        'is_digit': char.isdigit(), #angka atau bukan, karena angka
        'is_alpha': char.isalpha(), #dia huruf atau ga
        'bias': 1.0
    }

    # Check kata sebelum dia
    if i > 0:
        char_prev = text[i - 1]
        features.update({
            '-1:char': char_prev,
            '-1:is_upper': char_prev.isupper(),
            '-1:is_digit': char_prev.isdigit(),
            '-1:is_alpha': char_prev.isalpha(),
        })
    else:
        features['BOS'] = True  # Beginning of sequence, awal kata

    # Next character features
    if i < len(text) - 1:
        char_next = text[i + 1]
        features.update({
            '+1:char': char_next,
            '+1:is_upper': char_next.isupper(),
            '+1:is_digit': char_next.isdigit(),
            '+1:is_alpha': char_next.isalpha(),
        })
    else:
        features['EOS'] = True  # End of sequence, akhir kata

    return features

def text_to_features(text):
    return [char_features(text, i) for i in range(len(text))]

def text_to_labels(text):
    return text_to_bio(text)

In [ ]:
!pip install sklearn-crfsuite

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 14.1 MB/s eta 0:00:00


In [ ]:
from sklearn_crfsuite import CRF
from sklearn_crfsuite.metrics import flat_classification_report

dataset = load_model()
print(dataset)
#training dataset
train_texts = dataset["Text"]
train_labels = [text_to_labels(t) for t in train_texts]

X_train = [text_to_features(t) for t in train_texts]
y_train = train_labels

# Train CRF Model
crf_bio = CRF(algorithm='lbfgs', c1=0.1, c2=0.1, max_iterations=100)
crf_bio.fit(X_train, y_train)

        Id Sentiment                             Instagram Comment Text
0        1  negative          "Udah ada berlian malah milih batu nisan"
1        2  negative          "2 3 tutup botol muka lu kek kontol wkwk"
2        3  negative  "Mukalu kek aktor bokep bgst mending ngobat aj...
3        4  positive  Hatinya terbuat dari apa ya .. Mbaa gigi yang ...
4        5  positive                "Motivasi ku ka Maudy cantipp bngt"
...    ...       ...                                                ...
1745  1746  positive  Selalu suka sama via org nya jujur apa adanya ...
1746  1747  positive  Selalu suka sama via org nya jujur apa adanya ...
1747  1748  positive  Selalu suka sama via org nya jujur apa adanya ...
1748  1749  positive  Selalu suka sama via org nya jujur apa adanya ...
1749  1750  positive  Selalu suka sama via org nya jujur apa adanya ...

[1750 rows x 3 columns]
                                                   Text  Label
0               Udah ada berlian malah milih bat

CRF(algorithm='lbfgs', c1=0.1, c2=0.1, max_iterations=100)

In [ ]:
def bio_to_tokens(text, bio_tags):
    tokens = []
    token = ""

    for char, tag in zip(text, bio_tags):
        if tag == "B":
            if token:
                tokens.append(token)
            token = char
        elif tag == "I":
            token += char
        elif tag == "O":
            if token:
                tokens.append(token)
                token = ""
    #last run word masuk sini
    if token:
        tokens.append(token)

    return tokens

# Example
# tokens = bio_to_tokens(test_text, y_pred[0])
# print("Extracted Tokens: ", tokens)

In [ ]:
# test_text = "Geblek lo tata...cowo bgt dibela2in balikan...hadeww...ntar ditinggal lg nyalahin tuh cowo...padahal kitenya yg oon."
# test_text = "eh... Situ siapa??? Kids jaman now ya kebanyakan micin.. Mna kata aku yg ngina??? Bego dipelihara??????"
# test_text = "CONGRATS KAK ISYAN semoga karya nya apa yg dilakukan kakak sukses teruss"
# test_text = "Thank you juga, Syan! Seneng banget kemarin haha! Semoga setelah pandemi ini, semua rencana dilancarkan ya, Syan!"
# test_text = "i see beauty in everything mba gigi sehat selalu sama dede bayi nya"
# test_text = "Foto no edit edit ya saayy kamera nyaa bening,,, model fotonya pun juga bening gak perlu pakai filter tipu tipu"
# test_text = "Gosh, tiap kali lihat isyana mainin alat musik kaya lihat jiwanya beda. Merinding gila. Kaya kepribadiannya jadi serius bangett"


dataset = load_model()
# print(dataset)
dataset_baru = dataset.set_index('Text')
dataset_baru['Token'] = ""
# print(dataset_baru)
for data in dataset['Text']:
  # print(data)
  X_test = [text_to_features(data)]
  # print(X_test)
  y_pred = crf_bio.predict(X_test)
  tokens = bio_to_tokens(data, y_pred[0])
  dataset_baru.loc[data, 'Token'] = str(tokens)

print(dataset_baru)
dataset_baru.to_excel("Hasil_Prediksi.xlsx")
# print("Test Text: ", test_text)
# print("Predicted BIO: ", "".join(y_pred[0]))

        Id Sentiment                             Instagram Comment Text
0        1  negative          "Udah ada berlian malah milih batu nisan"
1        2  negative          "2 3 tutup botol muka lu kek kontol wkwk"
2        3  negative  "Mukalu kek aktor bokep bgst mending ngobat aj...
3        4  positive  Hatinya terbuat dari apa ya .. Mbaa gigi yang ...
4        5  positive                "Motivasi ku ka Maudy cantipp bngt"
...    ...       ...                                                ...
1745  1746  positive  Selalu suka sama via org nya jujur apa adanya ...
1746  1747  positive  Selalu suka sama via org nya jujur apa adanya ...
1747  1748  positive  Selalu suka sama via org nya jujur apa adanya ...
1748  1749  positive  Selalu suka sama via org nya jujur apa adanya ...
1749  1750  positive  Selalu suka sama via org nya jujur apa adanya ...

[1750 rows x 3 columns]
                                                    Label  \
Text                                              

In [ ]:
df = pd.DataFrame({"text": open("/content/Corpus-Kata-Dasar_used.txt").read().splitlines()})
df = df[~df['text'].str.contains(r'\s')]
df['text'].to_csv("/content/Corpus-Kata-Dasar.txt", index=False, header=False)
print(df.head())
df = df.set_index("text")

df1 = pd.DataFrame({"text": open("/content/ASa_unique_used.txt").read().splitlines()})
df1 = df1[~df1['text'].str.contains(r'\s')]
df1['text'].to_csv("/content/ASa_unique.txt", index=False, header=False)
print(df1.head())
df1 = df1.set_index("text")

    text
0      @
1   abis
2     ad
3   adlh
4  afaik
       text
0       nya
1   didalem
2    woyyyy
3  singkron
4   jadinya


In [ ]:
# from rapidfuzz import process
# from langdetect import detect_langs
# import langid

dataset = pd.read_excel("/content/Hasil_Prediksi.xlsx")
datanya = dataset['Token']
dataset['Lang']=""
for data in datanya:
  tokenized_sentence = ast.literal_eval(data)
  label = []
  for word in tokenized_sentence:
    if(word.lower() in df.index.str.lower()):
      label.append('id')
    elif(word.lower() in df1.index.str.lower()):
      label.append('id')
    else:
        label.append('en')
        print(word)
  dataset.loc[dataset['Token'] == data, 'Lang'] = str(label)

print(dataset)
dataset.to_excel("newest_data.xlsx")
print(label)

Go
head
Happy
wariorr
happy
couple
youtube
youtube
bully
God
bless
congrats
Bully
Failed
hate
comment
CONGRATS
haters
Zoo
PROUD
OF
YOU
Proud
of
you
God
bless
cute
caption
attitude
Before
and
after
please
nice
dance
and
live
the
remix
backstage
gentle
the
remix
the
remix
works
many
ways
is
than
our
happy
wedding
soundwave
good
vibes
upload
public
figure
Yess
feeling
comment
keep
bruuh
bully
perfect
strong
bully
see
beauty
everything
SO
boss
congrats
Thank
you
frame
style
fresh
post
update
keep
touch
caption
toxic
build
You
will
infotainment
prepare
finally
congrats
Perfect
upload
comment
yesss
glowing
HATE
kids
now
why
you
always
stunning
pretty
girl
YouTube
Student
life
standford
Oxford
hate
instastory
comment
swett
so
swett
Princess
prince
cute
endorse
skincare
good
luck
the
fest
even
folowers
bless
you
both
kick
Congrats
or
awareness
link
bio
younger
than
her
humble
happy
YouTube
live
live
humble
comment
Kids
now
and
Youtube
daily
activity
Stanford
meeting
instastory
Happy
birthday
c

In [ ]:
import sklearn_crfsuite
from sklearn_crfsuite import CRF
from sklearn_crfsuite.metrics import flat_classification_report

def extract_features(tokens, index):
    token = tokens[index]
    features = {
        'token': token.lower(),
        'is_alpha': token.isalpha(),
        'is_digit': token.isdigit(),
        'is_title': token.istitle(),
        'is_upper': token.isupper(),
        'contains_digit': any(char.isdigit() for char in token),
        'contains_alpha': any(char.isalpha() for char in token),
        'has_apostrophe': "'" in token,
        'token_length': len(token),
        'char_ngrams': [token[i:i+5] for i in range(len(token)-4)] if len(token) >= 5 else [token],
    }

    # Add neighboring tokens as context
    if index > 0:
        features['prev_token'] = tokens[index - 1].lower()
    else:
        features['prev_token'] = '<START>'

    if index < len(tokens) - 1:
        features['next_token'] = tokens[index + 1].lower()
    else:
        features['next_token'] = '<END>'

    return features


In [ ]:
def transform_data(token_sequences, label_sequences):
    X, y = [], []
    for idx, (tokens, labels) in enumerate(zip(token_sequences, label_sequences)):
        if len(tokens) != len(labels):
            print(f"\nError in sentence {idx}: token count ({len(tokens)}) != label count ({len(labels)})")
            print(f"Tokens ({len(tokens)}): {tokens}")
            print(f"Labels ({len(labels)}): {labels}")
            raise ValueError(f"Mismatch in sentence {idx}")
        X.append([extract_features(tokens, i) for i in range(len(tokens))])
        y.append(labels)
    return X, y

In [ ]:
from collections import Counter

In [ ]:
datanya = pd.read_excel("/content/newest_data.xlsx",index_col=False)
# print(datanya)
datanya["Token"] = datanya["Token"].apply(ast.literal_eval)
datanya["Lang"] = datanya["Lang"].apply(ast.literal_eval)
token = datanya["Token"]
lang = datanya["Lang"]
original_label_counts = Counter([label for labels in lang for label in labels])
print("Label Distribution:")
for label, count in original_label_counts.items():
    print(f"{label}: {count}")
# print(token)
# X, y = transform_data(token, lang)
# print(X_train)
# print(y_train)

Label Distribution:
id: 31591
en: 1899


In [ ]:
balanced_token_sequences = []
balanced_label_sequences = []
max_class = max(original_label_counts.values())
for tokens, labels in zip(token, lang):
    label_freq = Counter(labels)
    max_label = max(label_freq.values())
    repeat_factor = min(5, max_class // max_label)  # limit duplkat 3 kali
    for _ in range(repeat_factor):
        balanced_token_sequences.append(tokens)
        balanced_label_sequences.append(labels)
balanced_label_counts = Counter([label for labels in balanced_label_sequences for label in labels])
print("\nBalanced Label Distribution:")
for label, count in balanced_label_counts.items():
    print(f"{label}: {count}")


Balanced Label Distribution:
id: 157955
en: 9495


In [ ]:
X, y = transform_data(balanced_token_sequences, balanced_label_sequences)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
crf = CRF(algorithm='lbfgs', max_iterations=200, c1=0.1, c2=0.1)
crf.fit(X_train, y_train)

CRF(algorithm='lbfgs', c1=0.1, c2=0.1, max_iterations=200)

In [ ]:
y_pred = crf.predict(X_test)
y_test_flat = np.concatenate(y_test)
y_pred_flat = np.concatenate(y_pred)
overall_accuracy = accuracy_score(y_test_flat, y_pred_flat)
print(f"Overall Accuracy: {overall_accuracy:.4f}")
print("\nDetailed Classification Report:")
print(classification_report(y_test_flat, y_pred_flat))

Overall Accuracy: 0.9999

Detailed Classification Report:
              precision    recall  f1-score   support

          en       1.00      1.00      1.00      1895
          id       1.00      1.00      1.00     32870

    accuracy                           1.00     34765
   macro avg       1.00      1.00      1.00     34765
weighted avg       1.00      1.00      1.00     34765



In [ ]:
# test_tokens = ['proud', 'of', 'you', 'indonesia', 'god', 'bless', 'indonesia']
# test_tokens = ['aku','telat','meeting']
test_tokens = ['gak', 'usah', 'sok', 'jadi', 'high', 'class', 'mba', 'liat', 'komenmu', 'sendiri', 'memang', 'semua', 'artis', 'itu', 'sama', 'beda2', 'mbak', 'ada', 'yg', 'suka', 'curhat', 'di', 'ig', 'ada', 'yg', 'curhat', 'ke', 'temen', 'ada', 'yg', 'curhat', 'ke', 'tuhan', 'ngak', 'usah', 'trus', 'mulut', 'situ', 'maju', 'dulu', 'otaknya', 'belakangan', 'mikirnya', 'yg', 'curh']
X_test = [extract_features(test_tokens, i) for i in range(len(test_tokens))]
predicted_labels = crf.predict([X_test])[0]

In [ ]:
for token, label in zip(test_tokens, predicted_labels):
    print(f"{token}: {label}")

gak: id
usah: id
sok: id
jadi: id
high: en
class: en
mba: id
liat: id
komenmu: id
sendiri: id
memang: id
semua: id
artis: id
itu: id
sama: id
beda2: id
mbak: id
ada: id
yg: id
suka: id
curhat: id
di: id
ig: id
ada: id
yg: id
curhat: id
ke: id
temen: id
ada: id
yg: id
curhat: id
ke: id
tuhan: id
ngak: id
usah: id
trus: id
mulut: id
situ: id
maju: id
dulu: id
otaknya: id
belakangan: id
mikirnya: id
yg: id
curh: id


In [ ]:
# Masuk Lexical Normalization
import re
import os
import requests
import gzip
import shutil
import nltk
import random
import pickle
from collections import defaultdict
from nltk.util import ngrams
from nltk.tokenize import word_tokenize
import pandas as pd
from nltk.tokenize import TweetTokenizer

In [ ]:
import re
import pandas as pd
import nltk
from nltk.tokenize import TweetTokenizer

# Pastikan NLTK sudah diunduh
nltk.download('punkt')

# Load dataset
file_path = "/content/colloquial-indonesian-lexicon (1).csv"  # Ganti dengan path file Anda
df = pd.read_csv(file_path)
# df['context'] = df['context'].str.replace('"', '', regex=False)


print(df)
# Membuat kamus slang -> formal dari dataset
slang_dict = dict(zip(df['slang'], df['formal']))

# Karakter khusus untuk mengganti kata slang
SPECIAL_CHAR = "<SLANG>"

# Inisialisasi TweetTokenizer
tokenizer = TweetTokenizer()

# def reduce_repeated_chars(text):
#     """Mengurangi pengulangan huruf menjadi maksimal 2 karakter."""
#     return re.sub(r'(.)\1{2,}', r'\1\1', text)  # Mengganti lebih dari 2 huruf berulang menjadi 2

# def remove_quotes(text):
#     return re.sub(r'["“”]', '', text)

def normalize_reduplicated_words(text):
    text = re.sub(r'(\w+)[2"\'’](\w*)', r'\1-\1\2', text)  # Ubah kata2nya -> kata-katanya
    return text

def normalize_suffix(text):
    """Menormalisasi akhiran tidak baku (-ny, -xa, -’y) menjadi -nya."""
    text = re.sub(r'(\w+)(-ny|-xa|-’y|-\'y)', r'\1-nya', text)  # rumahxa -> rumahnya
    return text

def replace_slang_with_special(text, slang_dict):
    """Mengganti kata slang dengan karakter khusus dalam teks."""
    words = text.split()
    processed_words = [SPECIAL_CHAR if word.lower() in slang_dict else word for word in words]
    return " ".join(processed_words)

def normalize_text(text, slang_dict):
    """Mengganti slang dengan kata formal."""
    words = text.split()
    normalized_words = [slang_dict[word.lower()] if word.lower() in slang_dict else word for word in words]
    return " ".join(normalized_words)

def clean_token(token):
    """Menghapus hanya koma dan spasi dari token, menjaga tanda baca lainnya."""
    return token.replace(",", "").strip()

def tokenize_text(text):
    """Melakukan tokenisasi dan membersihkan token dari hanya koma serta spasi."""
    tokens = tokenizer.tokenize(text)
    cleaned_tokens = [clean_token(token) for token in tokens if clean_token(token)]  # Hapus token kosong
    return cleaned_tokens

# Terapkan pre-normalisasi pada dataset
df['context_prenormalize'] = df['context'].fillna("")
# df['context_prenormalize'] = df['context_prenormalize'].apply(remove_quotes)
df['context_prenormalize'] = df['context_prenormalize'].apply(normalize_reduplicated_words)
df['context_prenormalize'] = df['context_prenormalize'].apply(normalize_suffix)

# Lanjutkan dengan tokenisasi dan normalisasi slang
df['context_with_special'] = df['context_prenormalize'].apply(lambda x: replace_slang_with_special(x, slang_dict))
df['context_normalized'] = df['context_prenormalize'].apply(lambda x: normalize_text(x, slang_dict))
df['context_tokenized'] = df['context_with_special'].apply(tokenize_text)

# Simpan hasil ke file CSV baru
output_file = "colloquial-indonesian-lexicon-processed.xlsx"
df.to_excel(output_file, index=False)

# Tampilkan beberapa hasil pertama setelah diproses
print(df[['context', 'context_with_special', 'context_normalized', 'context_tokenized']].head())

print(f"\nDataset yang telah diproses telah disimpan sebagai: {output_file}")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


          slang         formal  In-dictionary  \
0          woww            wow              1   
1         aminn           amin              1   
2           met        selamat              1   
3        netaas        menetas              1   
4       keberpa       keberapa              0   
...         ...            ...            ...   
15001  gataunya  enggak taunya              0   
15002      gtau     enggak tau              0   
15003     gatau     enggak tau              0   
15004     fans2        fan-fan              0   
15005   gaharus   enggak harus              0   

                                                 context    category1  \
0                                                    wow     elongasi   
1      Selamat ulang tahun kakak tulus semoga panjang...     elongasi   
2      Met hari netaas kak!? Wish you all the best @t...    abreviasi   
3      Met hari netaas kak!? Wish you all the best @t...     afiksasi   
4                               Birthday yg ke

In [ ]:
tokenizer = TweetTokenizer()

# === 1️⃣ Load Dataset Slang dan Buat Translation Model ===
file_path = "/content/colloquial-indonesian-lexicon-processed.xlsx"
df = pd.read_excel(file_path)

# Membuat Translation Model dari dataset
translation_model = defaultdict(lambda: defaultdict(int))

for _, row in df.iterrows():
    slang = str(row['slang']).lower()
    formal = str(row['formal']).lower()
    translation_model[slang][formal] += 1

# Konversi ke probabilitas
for slang in translation_model:
    total_count = sum(translation_model[slang].values())
    for formal in translation_model[slang]:
        translation_model[slang][formal] /= total_count

# Simpan Translation Model agar bisa digunakan ulang
tm_model_path = "/content/translation_model.pkl"
with open(tm_model_path, "wb") as f:
    pickle.dump({key: dict(value) for key, value in translation_model.items()}, f)

print(f"Translation Model berhasil disimpan di {tm_model_path}")

# === 2️⃣ Melatih Language Model (LM) 5-Gram dari Dataset ===
print("Melatih model 5-gram dari dataset...")

# Gabungkan semua teks normalisasi sebagai korpus LM
lm_corpus = " ".join(df['context_normalized'].dropna().astype(str).tolist())
tokens = tokenizer.tokenize(lm_corpus.lower())

# Bangun model 5-gram
five_grams = list(ngrams(tokens, 5))
lm_model = defaultdict(dict)  # Mengubah struktur untuk menghindari PicklingError

for w1, w2, w3, w4, w5 in five_grams:
    context = (w1, w2, w3, w4)
    if w5 in lm_model[context]:
        lm_model[context][w5] += 1
    else:
        lm_model[context][w5] = 1

# Normalisasi probabilitas
for context in lm_model:
    total_count = sum(lm_model[context].values())
    for next_word in lm_model[context]:
        lm_model[context][next_word] /= total_count

# Konversi LM ke dictionary biasa agar bisa disimpan
lm_model_dict = {key: dict(value) for key, value in lm_model.items()}

# Simpan model LM agar bisa digunakan ulang
lm_model_path = "/content/language_model_5gram.pkl"
with open(lm_model_path, "wb") as f:
    pickle.dump(lm_model_dict, f)

print(f"Language Model 5-gram berhasil disimpan di {lm_model_path}")

# === 3️⃣ Implementasi Phrase-Based SMT (PBSMT) dengan LM 5-gram ===
def predict_next_word(context):
    """Memprediksi kata berikutnya berdasarkan 4 kata sebelumnya."""
    context_tuple = tuple(context.split()[-4:])
    if context_tuple in lm_model_dict:
        next_word_options = list(lm_model_dict[context_tuple].keys())
        next_word_probs = list(lm_model_dict[context_tuple].values())
        return next_word_options[next_word_probs.index(max(next_word_probs))]  # Pilih kata dengan probabilitas tertinggi
    return "[Tidak ditemukan]"

def translate_sentence_pbsmt(sentence):
    """Menerjemahkan slang menggunakan PBSMT dengan bantuan LM 5-gram"""
    words = tokenizer.tokenize(sentence.lower())  # Gunakan TweetTokenizer
    translated_words = []

    for i in range(len(words)):
        word = words[i]
        if word in translation_model:
            translations = list(translation_model[word].keys())
            probabilities = list(translation_model[word].values())
            translated_word = random.choices(translations, probabilities)[0]
        else:
            translated_word = word

        # Gunakan LM 5-gram jika ada cukup kata sebelumnya
        if i >= 4:
            context = " ".join(translated_words[-4:])  # Ambil 4 kata sebelumnya
            next_word = predict_next_word(context)
            if next_word != "[Tidak ditemukan]":
                translated_word = next_word

        translated_words.append(translated_word)

    return " ".join(translated_words)

# === 4️⃣ Terapkan PBSMT ke Dataset ===
df['context_pbsmt'] = df['context_normalized'].fillna("").apply(translate_sentence_pbsmt)

# Simpan hasil ke file baru
output_file = "/content/colloquial-indonesian-lexicon-PBSMT.csv"
df.to_csv(output_file, index=False)

print(f"Dataset dengan PBSMT berhasil disimpan di {output_file}")

Translation Model berhasil disimpan di /content/translation_model.pkl
Melatih model 5-gram dari dataset...
Language Model 5-gram berhasil disimpan di /content/language_model_5gram.pkl
Dataset dengan PBSMT berhasil disimpan di /content/colloquial-indonesian-lexicon-PBSMT.csv


In [ ]:
import pandas as pd
import nltk
from nltk.translate.bleu_score import corpus_bleu
from nltk.tokenize import TweetTokenizer

# Initialize tokenizer
tokenizer = TweetTokenizer()

# Load dataset with PBSMT output
file_path = "/content/colloquial-indonesian-lexicon-PBSMT.csv"
df = pd.read_csv(file_path)

# Extract reference (human-labeled) and hypothesis (model output)
references = df['context_normalized'].dropna().astype(str).tolist()  # Human reference
hypotheses = df['context_pbsmt'].dropna().astype(str).tolist()  # Model-generated sentences

# Tokenize sentences using TweetTokenizer
references_tokenized = [[tokenizer.tokenize(ref.lower())] for ref in references]  # List of lists
hypotheses_tokenized = [tokenizer.tokenize(hyp.lower()) for hyp in hypotheses]  # List

# Compute BLEU score
bleu_score = corpus_bleu(references_tokenized, hypotheses_tokenized)

print(f"BLEU Score: {bleu_score:.4f}")

BLEU Score: 0.8737


In [ ]:
test_sentence = "hade permpuan itu lg!!!!sakit jiwa,knp harus dia yg jd peran utama di film hantu jeruk purut,ky khabisan stok artis ja"

# Translate using PBSMT
translated_sentence = translate_sentence_pbsmt(test_sentence)

print(f"Input: {test_sentence}")
print(f"Model Output: {translated_sentence}")

Input: hade permpuan itu lg!!!!sakit jiwa,knp harus dia yg jd peran utama di film hantu jeruk purut,ky khabisan stok artis ja
Model Output: hade permpuan itu lagi ! ! ! sakit jiwa , kenapa harus dia yang jadi peran utama di film hantu jeruk purut , kaya khabisan stok artis saja


In [ ]:
#
from googletrans import Translator
import re

In [ ]:
def reduce_repeated_chars(word):
    # Mengurangi pengulangan karakter menjadi maksimal 2 kali
    return re.sub(r'(.)\1{2,}', r'\1\1', word)

def normalize_reduplication_with_2(word):
    # Mengubah kata ulang dengan angka 2 menjadi kata ulang formal
    return re.sub(r'(\w+)2$', r'\1-\1', word)

def normalize_reduplication_space(word):
    # Menambahkan tanda '-' pada kata ulang.
    words = word.split()
    if len(words) == 2 and words[0] == words[1]:
        return f"{words[0]}-{words[1]}"
    return word

def expand_contractions(word):
    # Memperluas kontraksi bahasa Inggris
    contractions = {"i'm": "i am", "you're": "you are", "it's": "it is"}
    return contractions.get(word.lower(), word)

def remove_prefix_nge(word):
    # Menghapus awalan 'nge-' dari kata
    return re.sub(r'^nge', '', word)

def normalize_suffix_nya(word):
# Menghapus akhiran '-nya' dan menambahkan 'the' di awal kata
    if word.endswith("nya"):
        return f"the {word[:-3]}"
    return word

def normalize_text_column(df, token_column, lang_column):
    def normalize_text(word, lang):
        if lang == 'id':
            word = reduce_repeated_chars(word)
            word = normalize_reduplication_with_2(word)
            word = normalize_reduplication_space(word)
            word = remove_prefix_nge(word)
            word = normalize_reduplicated_words(word)
            word = normalize_suffix(word)
            word = translate_sentence_pbsmt(word)
            print(word)
        elif lang == 'en':
            word = reduce_repeated_chars(word)
            word = expand_contractions(word)
            word = normalize_suffix_nya(word)
        return word

    df[token_column] = df.apply(lambda row: [normalize_text(word, lang) for word, lang in zip(row[token_column], row[lang_column])], axis=1)
    return df


In [ ]:
data = {"Token": [['anyiennnnggg', 'suaranya', 'ancur', 'banget', 'lebih', 'merdu', 'tukang', 'gorengan'], ['tolong', 'bantuin', 'dong']],
        "Language": [['id', 'id', 'id', 'id', 'id', 'id', 'id', 'id'], ['id', 'id', 'id']]}
df = pd.DataFrame(data)
normalized_df = normalize_text_column(df, "Token", "Language")
print(normalized_df)


anyienngg
suaranya
hancur
banget
lebih
merdu
tukang
gorengan
tolong
bantuin
dong
                                               Token  \
0  [anyienngg, suaranya, hancur, banget, lebih, m...   
1                            [tolong, bantuin, dong]   

                           Language  
0  [id, id, id, id, id, id, id, id]  
1                      [id, id, id]  


In [ ]:
def segment_text(text, model):
    words = text.split()
    segments = []
    current_segment = []


    X_test = [extract_features(words, i) for i in range(len(words))]
    predicted_labels = model.predict([X_test])[0]

    if isinstance(predicted_labels, np.ndarray):
        predicted_labels = predicted_labels.tolist()

    current_lang = predicted_labels[0] if predicted_labels else "unknown"

    for word, word_lang in zip(words, predicted_labels):
        if word_lang == current_lang:
            current_segment.append(word)
        else:
            if current_segment:
                segments.append((" ".join(current_segment), current_lang))
            current_segment = [word]
            current_lang = word_lang

    if current_segment:
        segments.append((" ".join(current_segment), current_lang))

    return segments

In [ ]:
def translate_segments(segments, target_lang="id"):
    translator = Translator()
    translated_text = []

    for segment, lang in segments:
        if lang != target_lang and lang != "unknown":
            translated = translator.translate(segment, src=lang, dest=target_lang).text
        else:
            translated = segment
        translated_text.append(translated)

    return " ".join(translated_text)

In [ ]:
df= pd.read_excel('/content/newest_data.xlsx')
df["Token"] = df["Token"].apply(ast.literal_eval)
df["Language"] = df["Lang"].apply(ast.literal_eval)
# print(type(data))
# thedata = pd.DataFrame({"Token":data.to_list(),"label":label.to_list()})
# print(thedata)

dataframe = normalize_text_column(df,"Token","Language")
label = df["Label"]
print(dataframe)
translated_dataset = []
for sentence in dataframe["Token"]:
  print(type(sentence))
  normal_string = " ".join(sentence)
  print(normal_string)
  code_mixed_text = normal_string
  # code_mixed_text = "proud of you indonesia god bless indonesia"
  # code_mixed_text = "gak usah sok jadi high class mba liat komenmu sendiri memang semua artis itu sama beda2 mbak ada yg suka curhat di ig ada yg curhat ke temen ada yg curhat ke tuhan ngak usah trus mulut situ maju dulu otaknya belakangan mikirnya yg curh"
  model = crf
  words = code_mixed_text.split()
  segments = segment_text(code_mixed_text, model)
  print(segments)
  translated_text = translate_segments(segments)
  print(type(translated_text))
  # print("Original Text:", code_mixed_text)
  # print("Segments:", segments)
  print("Translated Text:", translated_text)
  translated_dataset.append(translated_text)

translated = pd.DataFrame({"Translated": translated_dataset,"Label":label})
# {"Text": df["Cleaned_Text"],"Label": df["Sentiment"]}
print(translated)
translated.to_excel("Translated.xlsx")
# print(translated)


Streaming output truncated to the last 5000 lines.
Translated Text: hanya bisa melihat hujatan klean kasihan sih listy kontrak nya di copot di hujat memang dunia sangat kejam
<class 'list'>
jelek macam buntut kuali hahaha
[('jelek macam buntut kuali hahaha', 'id')]
<class 'str'>
Translated Text: jelek macam buntut kuali hahaha
<class 'list'>
kesihan gue sama orang otak dangkal kayak begini
[('kesihan gue sama orang otak dangkal kayak begini', 'id')]
<class 'str'>
Translated Text: kesihan gue sama orang otak dangkal kayak begini
<class 'list'>
bagusan pantat panci aing atuh anjir
[('bagusan pantat panci aing atuh anjir', 'id')]
<class 'str'>
Translated Text: bagusan pantat panci aing atuh anjir
<class 'list'>
sedih ya sekarang ngamen di kawinan orang mantan istri mah naik pesawat jet bhay pelakor n botakbego
[('sedih ya sekarang ngamen di kawinan orang mantan istri mah naik pesawat jet bhay pelakor n botakbego', 'id')]
<class 'str'>
Translated Text: sedih ya sekarang ngamen di kawinan o